In [7]:
import psycopg2
from psycopg2 import sql
import json
from datetime import datetime

# Thông tin kết nối PostgreSQL
DB_CONFIG = {
    "host": "localhost",
    "database": "news_db",
    "user": "thinhtruongdhth19@gmail.com",
    "password": "thinhtruong",  # Thay bằng mật khẩu của bạn
    "port": "5050"
}

def create_connection():
    """Tạo kết nối đến PostgreSQL database"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("Kết nối database thành công!")
        return conn
    except Exception as e:
        print(f"Lỗi khi kết nối database: {e}")
        return None

def create_articles_table(conn):
    """Tạo bảng articles nếu chưa tồn tại"""
    try:
        cursor = conn.cursor()
        
        create_table_query = """
        CREATE TABLE IF NOT EXISTS articles (
            id SERIAL PRIMARY KEY,
            title TEXT NOT NULL,
            url TEXT UNIQUE NOT NULL,
            summary TEXT,
            time TEXT,
            author TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        
        cursor.execute(create_table_query)
        conn.commit()
        print("Đã tạo bảng articles thành công!")
        
        # Tạo trigger để cập nhật updated_at
        trigger_query = """
        CREATE OR REPLACE FUNCTION update_timestamp()
        RETURNS TRIGGER AS $$
        BEGIN
            NEW.updated_at = NOW();
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
        
        DROP TRIGGER IF EXISTS update_articles_timestamp ON articles;
        CREATE TRIGGER update_articles_timestamp
        BEFORE UPDATE ON articles
        FOR EACH ROW EXECUTE FUNCTION update_timestamp();
        """
        
        cursor.execute(trigger_query)
        conn.commit()
        print("Đã tạo trigger cập nhật timestamp!")
        
    except Exception as e:
        print(f"Lỗi khi tạo bảng: {e}")
        conn.rollback()

def insert_articles(conn, articles):
    """Insert dữ liệu vào bảng articles"""
    try:
        cursor = conn.cursor()
        
        inserted_count = 0
        updated_count = 0
        
        for article in articles:
            # Kiểm tra xem bài viết đã tồn tại chưa bằng URL
            check_query = "SELECT id FROM articles WHERE url = %s"
            cursor.execute(check_query, (article['url'],))
            exists = cursor.fetchone()
            
            if exists:
                # Nếu tồn tại thì update
                update_query = """
                UPDATE articles 
                SET title = %s, summary = %s, time = %s, author = %s 
                WHERE url = %s
                """
                cursor.execute(update_query, (
                    article['title'],
                    article['summary'],
                    article['time'],
                    article['author'],
                    article['url']
                ))
                updated_count += 1
            else:
                # Nếu chưa tồn tại thì insert mới
                insert_query = """
                INSERT INTO articles (title, url, summary, time, author)
                VALUES (%s, %s, %s, %s, %s)
                """
                cursor.execute(insert_query, (
                    article['title'],
                    article['url'],
                    article['summary'],
                    article['time'],
                    article['author']
                ))
                inserted_count += 1
        
        conn.commit()
        print(f"Đã thêm {inserted_count} bài viết mới và cập nhật {updated_count} bài viết!")
        return True
        
    except Exception as e:
        print(f"Lỗi khi insert dữ liệu: {e}")
        conn.rollback()
        return False

def main():
    # Đọc dữ liệu đã làm sạch từ file JSON
    try:
        with open('cleaned_vnexpress_articles.json', 'r', encoding='utf-8') as f:
            articles = json.load(f)
    except Exception as e:
        print(f"Lỗi khi đọc file dữ liệu: {e}")
        return
    
    # Kết nối database
    conn = create_connection()
    if not conn:
        return
    
    try:
        # Tạo bảng nếu chưa tồn tại
        create_articles_table(conn)
        
        # Insert dữ liệu vào bảng
        insert_articles(conn, articles)
        
    finally:
        # Đóng kết nối
        if conn:
            conn.close()
            print("Đã đóng kết nối database")

if __name__ == "__main__":
    main()

In [4]:
pip install psycopg2-binary

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   ------------------ --------------------- 0.5/1.2 MB 262.1 kB/s eta 0:00:03
   ------------------ --------------------- 0.5/1.2 MB 262.1 kB/s eta 0:00:03
   ------------------ --------------------- 0.5/1.2 MB 262.1 kB/s eta 0:00:03
   ------------------ --------------------- 0.5/1.2 MB 262.1 kB/s eta 0:00:03
   --------------------------- ------------ 0.8/1.2 MB 286.7 kB/s eta 0:00:02
   --------------------------- ------------ 0.8/1.2 MB 286.7 kB/s eta 0:00:02
   --------------------------- ------------ 0.8/1